## **Settings**

- Importazione delle librerie necessarie e impostazione dei percorsi.

In [1]:
# Importazione delle librerie necessarie
import os
import sys
import random
from sys import platform

# Impostazione dei percorsi
_base_path = '\\'.join(os.getcwd().split('\\')[:-1]) + '\\' if platform == 'win32' else '/'.join(os.getcwd().split('/')[:-1]) + '/'
sys.path.append(_base_path)

# Importare le librerie necessarie
from monai.utils import set_determinism
from src.helpers.config import get_config
from src.models.gnn import GraphSAGE, GAT, ChebNet
from torch_geometric.explain import Explainer, ModelConfig, ThresholdConfig
# Rimuoviamo PGExplainer e usiamo solo GNNExplainer
from torch_geometric.explain.algorithm import GNNExplainer
import torch
import numpy as np
import matplotlib.pyplot as plt
import time

- Definizione dei percorsi per i dati, grafi, modelli salvati e report

In [2]:
# Definizione dei percorsi
_config = get_config()
data_path = os.path.join(_base_path, _config.get('DATA_FOLDER'))
graph_path = os.path.join(data_path, _config.get('GRAPH_FOLDER'))
saved_path = os.path.join(_base_path, _config.get('SAVED_FOLDER'))
reports_path = os.path.join(_base_path, _config.get('REPORT_FOLDER'))
logs_path = os.path.join(_base_path, _config.get('LOG_FOLDER'))

if platform == 'win32':
    data_path = data_path.replace('/', '\\')
    graph_path = graph_path.replace('/', '\\')
    saved_path = saved_path.replace('/', '\\')
    reports_path = reports_path.replace('/', '\\')
    logs_path = logs_path.replace('/', '\\')

- Impostazione dei seed per la riproducibilità

In [3]:
# Impostare un seed per la riproducibilità
set_determinism(seed=3)
random.seed(3)
np.random.seed(3)
torch.manual_seed(3)

## **Definizione del modello**

- Configurazione dei parametri del modello e creazione dell'istanza

In [4]:
# Definizione dei parametri del modello
# PARAMETRI CONDIVISI
num_node_features = 50          # Dimensione feature di input
num_classes = 4                 # Numero di classi di output
lr = 1e-4                       # Learning rate per l'ottimizzatore
weight_decay = 1e-5             # Weight decay per l'ottimizzatore
dropout = .0                    # Probabilità di dropout (per features)
hidden_channels = [512, 512, 512, 512, 512, 512, 512]  # Unità nascoste

# PARAMETRI GRAPHSAGE
aggr = 'mean'                   # Operazione di aggregazione

# PARAMETRI GAT
heads = 14                      # Numero di attention heads
attention_dropout = .1          # Probabilità di dropout (per attention)

# PARAMETRI CHEBNET
k = 4                           # Ordine polinomiale Chebyshev

# Creazione del modello da utilizzare
model = ChebNet(
    in_channels=num_node_features,
    hidden_channels=hidden_channels,
    out_channels=num_classes,
    dropout=dropout,
    K=k
)
print(f"Modello creato: {model.__class__.__name__}")

Modello creato: ChebNet


## **Caricamento dei dati**

- Funzione per la ricerca e il caricamento di un grafo

In [5]:
# Funzione per trovare e caricare un grafo per l'analisi
def find_and_load_graph(subject_id=None):
    """
    Trova e carica un grafo per l'analisi.
    
    Args:
        subject_id: ID specifico del soggetto da caricare, se None ne verrà scelto uno casualmente
        
    Returns:
        data: Il grafo caricato
        subject_id: L'ID del soggetto caricato
    """
    if subject_id is None:
        # Trova le cartelle dei soggetti che contengono grafi
        subject_dirs = [d for d in os.listdir(graph_path) if os.path.isdir(os.path.join(graph_path, d))]
        valid_subjects = []
        
        # Cerca i primi 10 soggetti che hanno file .graph
        for subject in subject_dirs[:100]:  # Limita la ricerca per efficienza
            graph_file = os.path.join(graph_path, subject, f"{subject}.graph")
            if os.path.isfile(graph_file):
                valid_subjects.append(subject)
                if len(valid_subjects) >= 10:
                    break
        
        if not valid_subjects:
            raise FileNotFoundError("Nessun grafo trovato nella directory data/graphs/")
        
        # Scegli un soggetto casuale
        subject_id = random.choice(valid_subjects)
    
    # Carica il grafo
    graph_file = os.path.join(graph_path, subject_id, f"{subject_id}.graph")
    if not os.path.isfile(graph_file):
        raise FileNotFoundError(f"File grafo non trovato per il soggetto {subject_id}")
    
    print(f"Caricamento grafo: {graph_file}")
          
    data = torch.load(graph_file, weights_only=False)
    
    return data, subject_id

- Caricamento di un grafo specifico con alta accuratezza

In [6]:
# Carica un grafo specifico con alta accuratezza (come visto nel precedente test)
subject_id = "BraTS-GLI-01166-000"  # Grafo con accuratezza 100%
try:
    data, subject_id = find_and_load_graph(subject_id)
    print(f"Grafo caricato con successo: {subject_id}")
    print(f"Numero di nodi: {data.x.shape[0]}")
    print(f"Numero di archi: {data.edge_index.shape[1]}")
    print(f"Numero di features per nodo: {data.x.shape[1]}")
except FileNotFoundError as e:
    print(f"Errore: {e}")
    print("Tentativo di caricamento di un grafo alternativo...")
    data, subject_id = find_and_load_graph(None)
    print(f"Grafo alternativo caricato: {subject_id}")

Caricamento grafo: C:\Users\gianluca\Desktop\brain-tumor-graph-segmentation-main\data\graphs\BraTS-GLI-01166-000\BraTS-GLI-01166-000.graph
Grafo caricato con successo: BraTS-GLI-01166-000
Numero di nodi: 2607
Numero di archi: 26070
Numero di features per nodo: 50


## **Caricamento del Modello Pre-addestrato**

- Ricerca e caricamento del modello ChebNet migliore

In [7]:
# Carica il modello pre-addestrato
model_files = [f for f in os.listdir(saved_path) if 'CHEBNET' in f and f.endswith('_best.pth')]
if not model_files:
    raise FileNotFoundError("Nessun modello ChebNet pre-addestrato trovato nella directory saved/")

latest_model = model_files[-1]
print(f"Utilizzo del modello pre-addestrato: {latest_model}")
model.load_state_dict(torch.load(os.path.join(saved_path, latest_model), map_location=torch.device('cpu')))
model.eval()

Utilizzo del modello pre-addestrato: CHEBNET_1739029370_best.pth


ChebNet(
  (layers): ModuleList(
    (0): ChebConv(50, 512, K=4, normalization=sym)
    (1-6): 6 x ChebConv(512, 512, K=4, normalization=sym)
    (7): ChebConv(512, 4, K=4, normalization=sym)
  )
  (dropout): Dropout(p=0.0, inplace=False)
)

## **Valutazione dell'Accuratezza**

- Verifica dell'accuratezza del modello sul grafo caricato.

In [8]:
# Verifica dell'accuratezza sul grafo caricato
with torch.no_grad():
    outputs = model(data.x, data.edge_index.type(torch.int64))
    predicted_labels = outputs.argmax(dim=1)
    
    # Visualizza le informazioni per debug
    print("\n--- DEBUG INFORMAZIONI DATI ---")
    print(f"Tipo di data.y: {type(data.y)}")
    if hasattr(data.y, 'shape'):
        print(f"Forma di data.y: {data.y.shape}")
    if hasattr(data.y, 'dtype'):
        print(f"Tipo di dati di data.y: {data.y.dtype}")
    print(f"Tipo di predicted_labels: {type(predicted_labels)}")
    print(f"Forma di predicted_labels: {predicted_labels.shape}")
    print(f"Tipo di dati di predicted_labels: {predicted_labels.dtype}")
    
    try:
        # Prova a estrarre il primo elemento di data.y per vedere se funziona
        if len(data.y) > 0:
            first_y = data.y[0]
            print(f"Primo elemento di data.y: {first_y}")
            if hasattr(first_y, 'shape'):
                print(f"Forma del primo elemento di data.y: {first_y.shape}")
    except Exception as e:
        print(f"Errore nell'accesso a data.y: {e}")
    
    print("--------------------------------\n")
    
    # Crea un tensore delle classi da usare, contenente le predizioni del modello
    node_classes = predicted_labels.clone()
    
    # Calcola l'accuratezza usando predicted_labels e data.y se possibile
    try:
        accuracy = (predicted_labels == data.y).float().mean().item()
        print(f"Accuratezza sul grafo {subject_id}: {accuracy:.4f}")
    except Exception as e:
        print(f"Impossibile calcolare l'accuratezza usando data.y: {e}")
        print("Usando solo le predizioni per l'analisi")
    
    class_counts = torch.bincount(predicted_labels, minlength=num_classes)
    print(f"Distribuzione classi predette: {class_counts.numpy()}")


--- DEBUG INFORMAZIONI DATI ---
Tipo di data.y: <class 'torch.Tensor'>
Forma di data.y: torch.Size([2607])
Tipo di dati di data.y: torch.float32
Tipo di predicted_labels: <class 'torch.Tensor'>
Forma di predicted_labels: torch.Size([2607])
Tipo di dati di predicted_labels: torch.int64
Primo elemento di data.y: 0.0
Forma del primo elemento di data.y: torch.Size([])
--------------------------------

Accuratezza sul grafo BraTS-GLI-01166-000: 1.0000
Distribuzione classi predette: [2550   16    4   37]


## **Implementazione di GNNExplainer**

- Configurazione e applicazione di GNNExplainer per la spiegabilità del modello

In [9]:
# Definizione dei nomi delle classi
# L'indice della lista corrisponde al valore numerico della classe.
# Assicurati che questo ordine sia coerente con le etichette nel tuo dataset BraTS
# e con l'output del tuo modello GNN.

classes = [
    "Sano/Background",      # Classe con indice numerico 0
    "NCR/NET",              # Classe con indice numerico 1 (Nucleo Necrotico/Non-Enhancing)
    "Edema (ED)",           # Classe con indice numerico 2
    "Tumore Enhancing (ET)" # Classe con indice numerico 3
]

In [ ]:
# ----- INIZIO CELLA 9 (O LA TUA CELLA DI SPIEGABILITÀ) -----

# Assicurati che le variabili `model`, `data`, `predicted_labels`, `num_node_features`, `classes`
# siano già definite e caricate correttamente dalle celle precedenti.

# 0. Importazioni necessarie (se non già presenti all'inizio del notebook)
import torch
import time
import random
import numpy as np
import matplotlib.pyplot as plt
from torch_geometric.explain import Explainer, ModelConfig # ThresholdConfig non lo usiamo subito qui
from torch_geometric.explain.algorithm import GNNExplainer

print("\n--- CONFIGURAZIONE E ESECUZIONE GNNEXPLAINER PER UN NODO SPECIFICO ---")

# 1. Selezionare un NODO TARGET da spiegare
target_class_value_for_explanation = 3  # Esempio: ET. Puoi cambiarlo.
node_to_explain_index = -1

# Controlla se predicted_labels esiste (dovrebbe dalla cella precedente)
if 'predicted_labels' not in locals() or not isinstance(predicted_labels, torch.Tensor):
    raise NameError("La variabile 'predicted_labels' non è definita. Esegui la cella dove calcoli le predizioni.")

candidate_indices = (predicted_labels == target_class_value_for_explanation).nonzero(as_tuple=True)[0]

if len(candidate_indices) > 0:
    node_to_explain_index = candidate_indices[random.randint(0, len(candidate_indices)-1)].item() # Scegli un candidato a caso
    print(f"Nodo target selezionato per la spiegazione: Indice {node_to_explain_index}")
    pred_class_idx = int(predicted_labels[node_to_explain_index].item())
    print(f"  Classe Predetta: {classes[pred_class_idx]} (Indice: {pred_class_idx})")
    if data.y is not None and node_to_explain_index < len(data.y):
        real_class_idx = int(data.y[node_to_explain_index].item())
        print(f"  Classe Reale:    {classes[real_class_idx]} (Indice: {real_class_idx})")
    else:
        print(f"  Classe Reale:    non disponibile per il nodo {node_to_explain_index}.")
else:
    if data.num_nodes > 0:
        node_to_explain_index = random.randint(0, data.num_nodes - 1) # Fallback
        print(f"Nessun nodo trovato per la classe target {target_class_value_for_explanation}. Si spiega il nodo casuale: Indice {node_to_explain_index}")
        pred_class_idx = int(predicted_labels[node_to_explain_index].item())
        print(f"  Classe Predetta per fallback: {classes[pred_class_idx]} (Indice: {pred_class_idx})")
        if data.y is not None and node_to_explain_index < len(data.y):
            real_class_idx = int(data.y[node_to_explain_index].item())
            print(f"  Classe Reale per fallback:    {classes[real_class_idx]} (Indice: {real_class_idx})")
        else:
            print(f"  Classe Reale per fallback:    non disponibile per il nodo {node_to_explain_index}.")
    else:
        raise ValueError("Il grafo caricato non ha nodi.")


# 2. Configurare l'Algoritmo GNNExplainer
gnn_explainer_algorithm = GNNExplainer(
    epochs=200,
    lr=0.01,
    coeffs={ # Questi coefficienti aiutano a ottenere spiegazioni più "pulite"
        "edge_size": 0.005,
        "node_feat_size": 1.0, # Meno penalità sull'uso delle feature per una spiegazione basata su feature
        "edge_ent": 1.0,
        "node_feat_ent": 0.1,
    }
    # return_type='raw' # Default è 'raw' che si aspetta logits.
                        # Se il tuo modello ha LogSoftmax, GNNExplainer può avere problemi a convergere.
                        # Potrebbe essere necessario passare l'output del modello (logits) direttamente
                        # o assicurarsi che `model_config.return_type` sia gestito correttamente.
)

# 3. Configurare l'Oggetto `Explainer` generale
model_config = ModelConfig(
    mode="multiclass_classification",
    task_level="node",
    return_type="log_probs",  # DEVE corrispondere all'output di model(data.x, data.edge_index)
                              # Se il modello restituisce logits, cambia questo in "raw"
)

explainer = Explainer(
    model=model, # Il tuo modello ChebNet pre-addestrato
    algorithm=gnn_explainer_algorithm,
    explanation_type="phenomenon",     # <-- Chiave per il tuo obiettivo!
    model_config=model_config,
    node_mask_type="attributes",       # Vogliamo importanza delle FEATURES
    edge_mask_type="object",           # Vogliamo importanza degli ARCHI (quindi vicini)
    # Non usare threshold_config qui per ora, analizza le maschere grezze.
)

# 4. Generare la Spiegazione per il Nodo Target Selezionato
print(f"\nInizio generazione spiegazione per il nodo: {node_to_explain_index}...")
start_time = time.time()

# Per spiegare la predizione del modello PER IL NODO SCELTO, usa predicted_labels
# Se volessi spiegare rispetto alla ground truth, useresti data.y
target_for_explanation = predicted_labels.type(torch.LongTensor) # Usa TUTTE le etichette (predette)
                                                                  # GNNExplainer userà 'index' per selezionare.

explanation = explainer(
    x=data.x,
    edge_index=data.edge_index.type(torch.int64), # Assicurati sia LongTensor
    index=node_to_explain_index,                  # Il NODO specifico da spiegare
    target=target_for_explanation                 # Il tensore completo delle etichette target (predette)
)

elapsed_time = time.time() - start_time
print(f"Spiegazione per il nodo {node_to_explain_index} generata in {elapsed_time:.2f} secondi.")

# 5. DEBUG Iniziale: Stampa informazioni sulle maschere ottenute
print("\n--- INFORMAZIONI SULLA SPIEGAZIONE GENERATA (DEBUG) ---")
if hasattr(explanation, 'node_mask') and explanation.node_mask is not None:
    print(f"Forma di explanation.node_mask: {explanation.node_mask.shape}")
    # Dovrebbe essere [N_nodi_nel_sottografo, num_node_features] o simile.
    # O [num_node_features] se l'algoritmo è ottimizzato per dare solo quelle del target.
    # Il tuo output precedente era [2607, 50], che è [data.num_nodes, num_node_features]
    # Questo significa che hai l'importanza delle features per OGNI nodo nel contesto della
    # spiegazione del nodo target.
    if explanation.node_mask.numel() > 0:
        # Estraiamo l'importanza delle features PER IL NODO TARGET
        target_node_feature_mask = explanation.node_mask[node_to_explain_index]
        print(f"Valori di explanation.node_mask PER NODO TARGET {node_to_explain_index} (prime 5 features): {target_node_feature_mask.squeeze()[:5]}")
else:
    print("explanation.node_mask è None o non presente.")

if hasattr(explanation, 'edge_mask') and explanation.edge_mask is not None:
    print(f"Forma di explanation.edge_mask: {explanation.edge_mask.shape}")
    # Dovrebbe essere [N_archi_nel_sottografo]
    if explanation.edge_mask.numel() > 0:
        print(f"Valori di explanation.edge_mask (prime 5 archi importanti): {explanation.edge_mask[:5]}")
else:
    print("explanation.edge_mask è None o non presente.")

# Indici dei nodi e archi del sottografo esplicativo (se GNNExplainer li popola)
if hasattr(explanation, 'node_idx') and explanation.node_idx is not None:
    print(f"Indici dei nodi nel sottografo esplicativo (explanation.node_idx): {explanation.node_idx}")
    print(f"  Numero di nodi nel sottografo: {len(explanation.node_idx)}")
else:
    print("explanation.node_idx non presente. Questo potrebbe significare che le maschere si riferiscono al grafo completo, ma con valori non nulli solo per il sottografo rilevante.")

if hasattr(explanation, 'edge_index') and explanation.edge_index is not None:
    print(f"Archi nel sottografo esplicativo (explanation.edge_index, forma {explanation.edge_index.shape}):")
    if explanation.edge_index.numel() > 0 :
        print(f"  (Primi 5 archi del sottografo, se presenti): \n{explanation.edge_index[:,:5]}")
    else:
        print("  Nessun arco restituito in explanation.edge_index.")
else:
    print("explanation.edge_index non presente.")

# ----- FINE CELLA 9 (O LA TUA CELLA DI SPIEGABILITÀ) -----


--- CONFIGURAZIONE E ESECUZIONE GNNEXPLAINER PER UN NODO SPECIFICO ---
Nodo target selezionato per la spiegazione: Indice 847
  Classe Predetta: Tumore Enhancing (ET) (Indice: 3)
  Classe Reale:    Tumore Enhancing (ET) (Indice: 3)

Inizio generazione spiegazione per il nodo: 847...


In [ ]:
# ----- INIZIO NUOVA CELLA PER PLOT FEATURES NODO TARGET -----

if hasattr(explanation, 'node_mask') and explanation.node_mask is not None and \
   explanation.node_mask.shape[0] == data.num_nodes and \
   explanation.node_mask.shape[1] == num_node_features:

    target_node_feature_importances = explanation.node_mask[node_to_explain_index].cpu().detach().numpy()

    print(f"\nVisualizzazione dell'Importanza delle {num_node_features} features per il NODO TARGET {node_to_explain_index}:")

    plt.figure(figsize=(15, 6))
    bar_positions = np.arange(num_node_features)
    plt.bar(bar_positions, target_node_feature_importances)
    plt.xlabel("Indice della Feature")
    plt.xticks(bar_positions[::5]) # Mostra un tick ogni 5 feature per leggibilità
    plt.ylabel("Importanza (da GNNExplainer)")
    plt.title(f"Importanza delle Features per Nodo Target {node_to_explain_index} (Classe Pred: {classes[predicted_labels[node_to_explain_index].item()]})")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()

    # Stampa le top K features più importanti
    k_top_features = 10
    # sorted_feature_indices = np.argsort(target_node_feature_importances)[::-1] # Decrescente
    sorted_feature_indices = np.flip(np.argsort(target_node_feature_importances)) # Corretto per evitare problemi di stride
    
    print(f"\nTop {k_top_features} features più importanti per il nodo {node_to_explain_index}:")
    # Se hai una lista `feature_names` di lunghezza `num_node_features`, usala qui!
    # Esempio: feature_names = [f"Feature_{i}" for i in range(num_node_features)]
    for i in range(min(k_top_features, num_node_features)):
        feat_idx = sorted_feature_indices[i]
        importance = target_node_feature_importances[feat_idx]
        # feat_name = feature_names[feat_idx] # Se hai feature_names
        print(f"  Feature {feat_idx}: Importanza = {importance:.4f}")
else:
    print("Formato di 'explanation.node_mask' non come atteso ([data.num_nodes, num_node_features]) per l'analisi delle feature del nodo target.")
    print("Oppure 'explanation.node_mask' è None.")

# ----- FINE NUOVA CELLA PER PLOT FEATURES NODO TARGET -----

In [ ]:
# ----- INIZIO CELLA PER ANALISI ARCHI E VICINI IMPORTANTI (FOCALIZZATA SUL TARGET) -----

if hasattr(explanation, 'edge_mask') and explanation.edge_mask is not None:
    edge_importances_np = explanation.edge_mask.cpu().detach().numpy()
    original_graph_edges_np = data.edge_index.cpu().detach().numpy() # Archi originali del grafo

    if original_graph_edges_np.shape[1] != len(edge_importances_np):
        print(f"ATTENZIONE: Il numero di archi in data.edge_index ({original_graph_edges_np.shape[1]}) "
              f"non corrisponde alla lunghezza di explanation.edge_mask ({len(edge_importances_np)}). L'analisi degli archi non può procedere correttamente.")
    else:
        print(f"\n--- Analisi degli Archi e Vicini Importanti per il NODO TARGET: {node_to_explain_index} ---")
        
        # 1. Estrarre tutti gli archi connessi al nodo target e la loro importanza
        #    (Questa parte è già nel tuo codice precedente e la riutilizziamo)
        connected_edges_info = [] # Lista di tuple: (importance, node_u, node_v)
        for i in range(original_graph_edges_np.shape[1]):
            node_u = int(original_graph_edges_np[0, i])
            node_v = int(original_graph_edges_np[1, i])
            importance = edge_importances_np[i]

            if node_u == node_to_explain_index or node_v == node_to_explain_index:
                connected_edges_info.append((importance, node_u, node_v))
        
        # 2. Ordina gli archi CONNESSI AL TARGET per importanza (decrescente)
        connected_edges_info.sort(key=lambda x: x[0], reverse=True)

        k_top_connected_edges_to_show = 15 # Quanti archi connessi al target mostrare
        
        print(f"\nTop {k_top_connected_edges_to_show} archi più importanti CONNESSI al nodo target {node_to_explain_index}:")
        print(f"{'Arco (u, v)':<15} {'Importanza':<15} {'Classe Nodo U':<25} {'Classe Nodo V':<25}")
        print("-" * 80)

        if not connected_edges_info:
            print(f"Nessun arco con importanza > 0 trovato direttamente connesso al nodo target {node_to_explain_index}.")
        else:
            important_neighbors_of_target = {}
            # Mostra solo i top k_top_connected_edges_to_show archi *che sono connessi al target*
            for i in range(min(k_top_connected_edges_to_show, len(connected_edges_info))):
                importance, node_u, node_v = connected_edges_info[i]
                
                # Assicurati che predicted_labels e classes siano definiti
                class_u_idx = int(predicted_labels[node_u].item())
                class_v_idx = int(predicted_labels[node_v].item())
                class_u_str = classes[class_u_idx]
                class_v_str = classes[class_v_idx]
                
                print(f"({node_u}-{node_v}){'':<5} {importance:<15.4f} {class_u_str:<25} {class_v_str:<25}")

                # Identifica il vicino e aggrega la sua importanza
                neighbor_node_idx = node_v if node_u == node_to_explain_index else node_u
                important_neighbors_of_target[neighbor_node_idx] = important_neighbors_of_target.get(neighbor_node_idx, 0) + importance
            
            if important_neighbors_of_target:
                print(f"\nTop Vicini più influenti per il nodo target {node_to_explain_index} (basato su importanza archi connessi):")
                # Ordina i vicini per importanza aggregata
                sorted_direct_neighbors = sorted(important_neighbors_of_target.items(), key=lambda item: item[1], reverse=True)
                for neighbor_idx_int, total_importance in sorted_direct_neighbors[:10]: # Mostra i top 10 vicini
                    class_neighbor_idx = int(predicted_labels[neighbor_idx_int].item())
                    print(f"  Vicino {neighbor_idx_int} (Classe Predetta: {classes[class_neighbor_idx]}): Importanza Aggreg. Archi = {total_importance:.4f}")
            else:
                print(f"Nessun vicino significativo identificato per il nodo {node_to_explain_index} dai top archi connessi.")

        # ----- Inizio Sezione Visualizzazione NetworkX (OPZIONALE) -----
        if important_neighbors_of_target: # Visualizza solo se ci sono vicini importanti
            try:
                import networkx as nx
                # import matplotlib.pyplot as plt # Assicurati sia importato
                
                plt.figure(figsize=(12, 10))
                vis_graph = nx.Graph()
                
                # Nodi da includere nella visualizzazione: il target e i suoi vicini importanti
                nodes_to_visualize_set = {node_to_explain_index}.union(set(important_neighbors_of_target.keys()))
                
                node_labels_vis = {}
                node_color_list_vis = []
                
                # Palette di colori (definisci 'discrete_colors_for_classes' come suggerito prima)
                cmap_for_plot = plt.get_cmap('viridis')
                discrete_colors_for_classes = [cmap_for_plot(i / (num_classes - 1)) for i in range(num_classes)]

                for node_idx in list(nodes_to_visualize_set):
                    vis_graph.add_node(node_idx)
                    pred_class_idx = int(predicted_labels[node_idx].item())
                    node_labels_vis[node_idx] = f"{node_idx}\n({classes[pred_class_idx]})"
                    if node_idx == node_to_explain_index:
                        node_color_list_vis.append('red')
                    elif 0 <= pred_class_idx < num_classes:
                        node_color_list_vis.append(discrete_colors_for_classes[pred_class_idx])
                    else:
                        node_color_list_vis.append('grey') # Fallback

                # Archi da disegnare: solo quelli tra il nodo target e i suoi vicini importanti
                # (che sono già in `connected_edges_info` e filtrati da `important_neighbors_of_target`)
                edges_to_visualize = []
                edge_weights_visualize = []

                for importance_val, u, v in connected_edges_info: # Itera sugli archi già connessi al target e ordinati
                    if u in nodes_to_visualize_set and v in nodes_to_visualize_set: # Assicura che entrambi siano nel nostro set
                        # Limita al numero di top archi connessi da visualizzare
                        if len(edges_to_visualize) < k_top_connected_edges_to_show :
                            edges_to_visualize.append((u, v))
                            edge_weights_visualize.append(max(0.1, importance_val * 7)) # Scala importanza per spessore
                        else:
                            break 
                
                vis_graph.add_edges_from(edges_to_visualize)

                if vis_graph.number_of_nodes() > 0:
                    pos = nx.spring_layout(vis_graph, k=0.7, iterations=40)
                    nx.draw(vis_graph, pos, labels=node_labels_vis, with_labels=True, 
                            node_color=node_color_list_vis, node_size=1500, font_size=9, 
                            width=edge_weights_visualize, font_color='black', edge_color='darkgrey',
                            alpha=0.9)
                    plt.title(f"Sottografo Locale Esplicativo per Nodo {node_to_explain_index} (Rosso=Target)", fontsize=15)
                    plt.show()
                else:
                    print(f"Sottografo per la visualizzazione è vuoto per il nodo {node_to_explain_index}.")

            except ImportError:
                print("NetworkX non è installato. Salta la visualizzazione del sottografo dei vicini.")
            except Exception as e_vis:
                print(f"Errore durante la visualizzazione del sottografo dei vicini: {e_vis}")
        # ----- Fine Sezione Visualizzazione NetworkX -----
else:
    print("Mancano `explanation.edge_mask` per l'analisi dettagliata degli archi.")

# ----- FINE CELLA PER ANALISI ARCHI E VICINI IMPORTANTI (FOCALIZZATA SUL TARGET) -----

In [ ]:
# ----- INIZIO NUOVA CELLA PER ANALISI ARCHI E VICINI IMPORTANTI -----

# Visualizzazione del sottografo importante (più avanzato, richiede NetworkX)
# e analisi degli archi.
# Dobbiamo usare `explanation.edge_mask` e `explanation.edge_index` (del sottografo)

if hasattr(explanation, 'edge_mask') and explanation.edge_mask is not None and \
   hasattr(explanation, 'edge_index') and explanation.edge_index is not None:

    edge_importances_np = explanation.edge_mask.cpu().detach().numpy()
    subgraph_edges_np = explanation.edge_index.cpu().detach().numpy() # Forma [2, NumArchiSottografo]

    if subgraph_edges_np.shape[1] != len(edge_importances_np):
        print(f"ATTENZIONE: Il numero di archi in explanation.edge_index ({subgraph_edges_np.shape[1]}) "
              f"non corrisponde alla lunghezza di explanation.edge_mask ({len(edge_importances_np)}).")
        print("L'analisi degli archi potrebbe non essere corretta.")
    else:
        print(f"\nAnalisi degli archi più importanti NEL SOTTOGRAFO ESPLICATIVO per il nodo {node_to_explain_index}:")
        k_top_edges_to_show = 15
        # sorted_edge_indices_in_subgraph = np.argsort(edge_importances_np)[::-1]
        sorted_edge_indices_in_subgraph = np.flip(np.argsort(edge_importances_np))


        print(f"{'Arco Sottografo (u_sub,v_sub)':<25} {'Importanza':<15} {'Nodo Orig. u':<15} {'Classe u':<20} {'Nodo Orig. v':<15} {'Classe v':<20}")
        print("-" * 110)

        # `explanation.node_idx` mappa gli indici del sottografo a quelli originali.
        # Se `explanation.node_idx` è None, si assume che subgraph_edges_np usi già indici originali (improbabile)
        # o che non si possa fare la mappatura diretta facilmente.
        # Il tuo ultimo output diceva "explanation.node_idx non presente."
        # Questo è un problema per mappare i nodi del sottografo a quelli originali!

        # CONTROLLIAMO SE `explanation` CONTIENE GLI INDICI DEI NODI DEL SOTTOGRAFO
        subgraph_node_indices_original = None
        if hasattr(explanation, 'node_idx') and explanation.node_idx is not None:
            subgraph_node_indices_original = explanation.node_idx.cpu().detach().numpy()
            print(f"Trovato explanation.node_idx con {len(subgraph_node_indices_original)} nodi.")
        else:
            print("ATTENZIONE: `explanation.node_idx` non è disponibile. Non è possibile mappare gli indici dei nodi del sottografo agli indici originali in modo affidabile con queste informazioni.")
            print("L'analisi dei 'vicini importanti' sarà limitata o potrebbe essere errata.")
            # Se questo succede, `subgraph_edges_np` potrebbe contenere indici originali
            # e il "sottografo" è l'intero grafo, con `edge_mask` che rende la maggior parte degli archi non importanti.
            # O, `subgraph_edges_np` è un sottografo ma non sappiamo a quali nodi originali si riferisce.

        important_neighbors_of_target = {}

        for i in range(min(k_top_edges_to_show, len(sorted_edge_indices_in_subgraph))):
            idx_in_mask_and_subgraph_edges = sorted_edge_indices_in_subgraph[i]
            importance = edge_importances_np[idx_in_mask_and_subgraph_edges]
            
            # Nodi u e v come appaiono in subgraph_edges_np
            # Questi sono indici RELATIVI ai nodi presenti in subgraph_node_indices_original,
            # se subgraph_node_indices_original è definito.
            node_u_sub_idx = subgraph_edges_np[0, idx_in_mask_and_subgraph_edges]
            node_v_sub_idx = subgraph_edges_np[1, idx_in_mask_and_subgraph_edges]

            # Mappa a indici originali se possibile
            node_u_orig, node_v_orig = "?", "?"
            class_u_str, class_v_str = "N/A", "N/A"

            if subgraph_node_indices_original is not None:
                if node_u_sub_idx < len(subgraph_node_indices_original):
                    node_u_orig = subgraph_node_indices_original[node_u_sub_idx]
                    class_u_str = classes[predicted_labels[node_u_orig].item()]
                if node_v_sub_idx < len(subgraph_node_indices_original):
                    node_v_orig = subgraph_node_indices_original[node_v_sub_idx]
                    class_v_str = classes[predicted_labels[node_v_orig].item()]
            else:
                # Se node_idx non c'è, proviamo a interpretare u e v come indici originali
                # Questo è probabile se GNNExplainer ha dato una edge_mask per TUTTI gli archi originali
                # e explanation.edge_index è semplicemente data.edge_index (ma GNNExplainer di solito lo filtra).
                # Questo caso è meno probabile con la nuova API Explainer per "phenomenon".
                # Assumiamo che gli indici in subgraph_edges_np SIANO originali se node_idx non c'è
                # (con cautela, perché potrebbe essere errato).
                node_u_orig = node_u_sub_idx
                node_v_orig = node_v_sub_idx
                if node_u_orig < data.num_nodes: class_u_str = classes[predicted_labels[node_u_orig].item()]
                if node_v_orig < data.num_nodes: class_v_str = classes[predicted_labels[node_v_orig].item()]


            is_direct_neighbor_of_target = False
            if node_u_orig == node_to_explain_index or node_v_orig == node_to_explain_index:
                is_direct_neighbor_of_target = True
                
                neighbor_node_idx = node_v_orig if node_u_orig == node_to_explain_index else node_u_orig
                if isinstance(neighbor_node_idx, int): # Assicurati che sia un indice valido
                     important_neighbors_of_target[neighbor_node_idx] = important_neighbors_of_target.get(neighbor_node_idx, 0) + importance

            direct_flag = "*" if is_direct_neighbor_of_target else " "
            print(f"{direct_flag}({node_u_sub_idx}-{node_v_sub_idx}){'':<5} {importance:<15.4f} {str(node_u_orig):<15} {class_u_str:<20} {str(node_v_orig):<15} {class_v_str:<20}")

        if important_neighbors_of_target:
            print(f"\nVicini più importanti del nodo target {node_to_explain_index} (e la loro importanza aggregata dagli archi):")
            sorted_direct_neighbors = sorted(important_neighbors_of_target.items(), key=lambda item: item[1], reverse=True)
            for neighbor, total_importance in sorted_direct_neighbors[:10]:
                print(f"  Vicino {neighbor} (Classe: {classes[predicted_labels[neighbor].item()]}): Imp. Aggregata Archi = {total_importance:.4f}")

        # (La visualizzazione con NetworkX andrebbe qui, se decidi di implementarla)

else:
    print("Mancano edge_mask o edge_index per analizzare gli archi importanti.")

# ----- FINE NUOVA CELLA PER ANALISI ARCHI E VICINI IMPORTANTI -----